## Evaluating the suitability filter on RxRx1

In [1]:
import importlib
import random

import numpy as np
import pandas as pd
import torch

from suitability.datasets.wilds import get_wilds_dataset, get_wilds_model
from suitability.filter import suitability_efficient

importlib.reload(suitability_efficient)

from suitability.filter.suitability_efficient import SuitabilityFilter, get_sf_features

# Set seeds for reproducibility
random.seed(32)
np.random.seed(32)

### Define & evaluate all possible splits

In [2]:
val_splits = [
    ("val", {"cell_type": "HEPG2"}),
    ("val", {"cell_type": "HUVEC"}),
    ("val", {"cell_type": "RPE"}),
    ("val", {"cell_type": "U2OS"}),
    ("val", {})
]

id_test_splits = [
    ("id_test", {"cell_type": "HEPG2"}),
    ("id_test", {"cell_type": "HUVEC"}),
    ("id_test", {"cell_type": "RPE"}),
    ("id_test", {"cell_type": "U2OS"}),
    ("id_test", {})
]

test_splits = [
    ("test", {"cell_type": "HEPG2"}),
    ("test", {"cell_type": "HUVEC"}),
    ("test", {"cell_type": "RPE"}),
    ("test", {"cell_type": "U2OS"}),
    ("test", {})
]

In [ ]:
data_name = "rxrx1"
root_dir = "/mfsnic/u/apouget/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = get_wilds_model(data_name, root_dir, algorithm="ERM")
model = model.to(device)
model.eval()

all_splits = val_splits + test_splits + id_test_splits

results = pd.DataFrame(columns=["split", "cell_type", "num_samples", "accuracy"])

for split, pre_filter in all_splits:
    data = get_wilds_dataset(
        data_name,
        root_dir,
        split,
        batch_size=64,
        shuffle=False,
        num_workers=4,
        pre_filter=pre_filter,
    )
    features, corr = get_sf_features(data, model, device)

    num_samples = len(data.dataset)
    accuracy = np.mean(corr)
    cell_type = pre_filter.get("cell_type", "ALL")
    results = results._append(
        {
            "split": split,
            "cell_type": cell_type,
            "num_samples": num_samples,
            "accuracy": accuracy,
        },
        ignore_index=True,
    )

results.to_csv("suitability/results/data_splits/rxrx1_ERM_0_last.csv", index=False)

/mfsnic/u/apouget/envs/test/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mfsnic/u/apouget/envs/test/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_3048830/3639878652.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result

## Evaluate suitability filter

In [2]:
import os
import pickle
import random

import numpy as np
import pandas as pd
import torch

from suitability.datasets.wilds import get_wilds_dataset, get_wilds_model
from suitability.filter.suitability_efficient import SuitabilityFilter, get_sf_features

# Set seeds for reproducibility
random.seed(32)
np.random.seed(32)

# Configuration
data_name = "rxrx1"
root_dir = "/mfsnic/u/apouget/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
algorithm = "groupDRO"
model_type = "last"
seed = 0
model = get_wilds_model(
    data_name, root_dir, algorithm=algorithm, seed=seed, model_type=model_type
)
model = model.to(device)
model.eval()
print(f"Model loaded to device: {device}")

# Initialize results DataFrame
features_cache_file = (
    f"suitability/results/features/{data_name}_{algorithm}_{model_type}_{seed}.pkl"
)
valid_splits = ["val", "test", "id_test"]
splits_features_cache = {}

# Precompute all data features
for split_name in valid_splits:
    print(f"Computing features for split: {split_name}")
    dataset = get_wilds_dataset(
        data_name,
        root_dir,
        split_name,
        batch_size=64,
        shuffle=False,
        num_workers=4,
    )
    splits_features_cache[split_name] = get_sf_features(dataset, model, device)
print("Features computed")

# Save feature cache
with open(features_cache_file, "wb") as f:
    pickle.dump(splits_features_cache, f)

# Precompute all id split indices
cache_file = f"suitability/results/split_indices/{data_name}.pkl"

valid_splits = [
    ("val", {"cell_type": "HEPG2"}),
    ("val", {"cell_type": "HUVEC"}),
    ("val", {"cell_type": "RPE"}),
    ("val", {"cell_type": "U2OS"}),
    ("id_test", {"cell_type": "HEPG2"}),
    ("id_test", {"cell_type": "HUVEC"}),
    ("id_test", {"cell_type": "RPE"}),
    ("id_test", {"cell_type": "U2OS"}),
    ("test", {"cell_type": "HEPG2"}),
    ("test", {"cell_type": "HUVEC"}),
    ("test", {"cell_type": "RPE"}),
    ("test", {"cell_type": "U2OS"}),
]

splits_indices_cache = {}
for split_name, split_filter in valid_splits:
    print(f"Computing indices for split: {split_name} with filter: {split_filter}")
    dataset, indices = get_wilds_dataset(
        data_name,
        root_dir,
        split_name,
        batch_size=64,
        shuffle=False,
        num_workers=4,
        pre_filter=split_filter,
        return_indices=True,
    )
    splits_indices_cache[(split_name, str(split_filter))] = indices

with open(cache_file, "wb") as f:
    pickle.dump(splits_indices_cache, f)

print("splits indices computed")

/mfsnic/u/apouget/envs/test/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mfsnic/u/apouget/envs/test/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded to device: cuda
Computing features for split: val
Computing features for split: test
Computing features for split: id_test
Features computed


# OOD SPLIT SUBSET EVALS

In [ ]:
import pickle
import random

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

from suitability.filter.suitability_efficient import SuitabilityFilter
from suitability.filter.tests import non_inferiority_ttest

# Set seeds for reproducibility
random.seed(32)
np.random.seed(32)

# Configuration
data_name = "rxrx1"
root_dir = "/mfsnic/projects/suitability/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
algorithm = "ERM"
model_type = "last"
seed = 2

# Load the features
feature_cache_file = (
    f"suitability/results/features/{data_name}_{algorithm}_{model_type}_{seed}.pkl"
)
with open(feature_cache_file, "rb") as f:
    full_feature_dict = pickle.load(f)

# Load the split indices
split_cache_file = f"suitability/results/split_indices/{data_name}.pkl"
with open(split_cache_file, "rb") as f:
    ood_split_dict = pickle.load(f)

all_keys = list(ood_split_dict.keys())
for key in all_keys:
    if key[0] == 'id_test':
        del ood_split_dict[key]

# Define suitability filter and experiment parameters
classifiers = [
    "logistic_regression"
]  # "logistic_regression", "svm", "random_forest", "gradient_boosting", "mlp", "decision_tree"]
margins = [0, 0.005, 0.01, 0.05]
normalize = True
calibrated = True
sf_results = []
direct_testing_results = []
feature_subsets = [
    [0],
    [1],
    [2],
    [3],
    [4],
    [5],
    [6],
    [7],
    [8],
    [9],
    [10],
    [11],
    # [4, 11],
    # [4, 11, 8],
    # [4, 11, 8, 6],
    # [4, 11, 8, 6, 2],
    # [4, 11, 8, 6, 2, 1],
    # [4, 11, 8, 6, 2, 1, 0],
    # [4, 11, 8, 6, 2, 1, 0, 7],
    # [4, 11, 8, 6, 2, 1, 0, 7, 3],
    # [4, 11, 8, 6, 2, 1, 0, 7, 3, 10],
    # [4, 11, 8, 6, 2, 1, 0, 7, 3, 10, 5],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
]
num_fold_arr = [15]

source_features, source_corr = full_feature_dict["id_test"]


# Main loop
for user_split_name, user_filter in tqdm(ood_split_dict.keys()):
    print(f"Evaluating user split: {user_split_name} with filter {user_filter}")

    # Get user split indices
    user_split_indices = ood_split_dict[(user_split_name, user_filter)]

    # Get user split features and correctness
    all_features, all_corr = full_feature_dict[user_split_name]
    user_features = all_features[user_split_indices]
    user_corr = all_corr[user_split_indices]
    user_size = len(user_corr)
    user_acc = np.mean(user_corr)

    for num_folds in num_fold_arr:
        source_fold_size = len(source_corr) // num_folds
        indices = np.arange(len(source_corr))
        np.random.shuffle(indices)
        fold_indices = [
            indices[i * source_fold_size : (i + 1) * source_fold_size]
            for i in range(num_folds)
        ]

        for i, reg_indices in enumerate(fold_indices):
            reg_features = source_features[reg_indices]
            reg_corr = source_corr[reg_indices]
            reg_size = len(reg_corr)
            reg_acc = np.mean(reg_corr)

            for j, test_indices in enumerate(fold_indices):
                if i == j:
                    continue
                test_features = source_features[test_indices]
                test_corr = source_corr[test_indices]
                test_size = len(test_corr)
                test_acc = np.mean(test_corr)

                for classifier in classifiers:
                    for feature_subset in feature_subsets:
                        suitability_filter = SuitabilityFilter(
                            test_features,
                            test_corr,
                            reg_features,
                            reg_corr,
                            device,
                            normalize=normalize,
                            feature_subset=feature_subset,
                        )
                        suitability_filter.train_classifier(
                            calibrated=calibrated, classifier=classifier
                        )

                        for margin in margins:
                            # Test suitability filter
                            sf_test = suitability_filter.suitability_test(
                                user_features=user_features, margin=margin
                            )
                            p_value = sf_test["p_value"]
                            ground_truth = user_acc >= test_acc - margin

                            sf_results.append(
                                {
                                    "data_name": data_name,
                                    "algorithm": algorithm,
                                    "seed": seed,
                                    "model_type": model_type,
                                    "normalize": normalize,
                                    "calibrated": calibrated,
                                    "margin": margin,
                                    "reg_fold": i,
                                    "reg_size": reg_size,
                                    "reg_acc": reg_acc,
                                    "test_fold": j,
                                    "test_size": test_size,
                                    "test_acc": test_acc,
                                    "user_split": user_split_name,
                                    "user_filter": user_filter,
                                    "user_size": user_size,
                                    "user_acc": user_acc,
                                    "p_value": p_value,
                                    "ground_truth": ground_truth,
                                    "classifier": classifier,
                                    "feature_subset": feature_subset,
                                    "acc_diff": user_acc - test_acc,
                                    "acc_diff_adjusted": user_acc + margin - test_acc,
                                }
                            )

                            # Run non-inferiority test on features directly
                            if (
                                len(feature_subset) == 1
                                and margin == 0
                                and classifier == "logistic_regression"
                                and (j == 0 or (i == 0 and j == 1))
                            ):
                                test_feature_subset = test_features[:, feature_subset].flatten()
                                user_feature_subset = user_features[:, feature_subset].flatten()
                                test_1 = non_inferiority_ttest(
                                    test_feature_subset,
                                    user_feature_subset,
                                    increase_good=True,
                                )
                                test_2 = non_inferiority_ttest(
                                    test_feature_subset,
                                    user_feature_subset,
                                    increase_good=False,
                                )
                                direct_testing_results.append(
                                    {
                                        "data_name": data_name,
                                        "algorithm": algorithm,
                                        "seed": seed,
                                        "model_type": model_type,
                                        "test_fold": j,
                                        "test_size": test_size,
                                        "test_acc": test_acc,
                                        "user_split": user_split_name,
                                        "user_filter": user_filter,
                                        "user_size": user_size,
                                        "user_acc": user_acc,
                                        "p_value_increase_good": test_1["p_value"],
                                        "p_value_decrease_good": test_2["p_value"],
                                        "ground_truth": ground_truth,
                                        "feature_subset": feature_subset,
                                        "acc_diff": user_acc - test_acc,
                                    }
                                )

# Save results
sf_evals = pd.DataFrame(sf_results)
sf_evals.to_csv(
    f"suitability/results/sf_evals/erm/rxrx1_sf_results_ood_subsets_{algorithm}_{model_type}_{seed}.csv",
    index=False,
)
direct_testing_evals = pd.DataFrame(direct_testing_results)
direct_testing_evals.to_csv(
    f"suitability/results/sf_evals/erm/rxrx1_direct_testing_results_ood_subsets_{algorithm}_{model_type}_{seed}.csv",
    index=False,
)


  0%|                                                                                                                            | 0/8 [00:00<?, ?it/s]

Evaluating user split: val with filter {'cell_type': 'HEPG2'}


 12%|██████████████▍                                                                                                    | 1/8 [01:49<12:44, 109.16s/it]

Evaluating user split: val with filter {'cell_type': 'HUVEC'}


 25%|████████████████████████████▊                                                                                      | 2/8 [03:37<10:53, 108.97s/it]

...

Evaluating user split: test with filter {'cell_type': 'HEPG2'}


 62%|███████████████████████████████████████████████████████████████████████▉                                           | 5/8 [09:03<05:27, 109.03s/it]

Evaluating user split: test with filter {'cell_type': 'HUVEC'}


 75%|██████████████████████████████████████████████████████████████████████████████████████▎                            | 6/8 [11:04<03:46, 113.02s/it]

Evaluating user split: test with filter {'cell_type': 'RPE'}


# ID SPLIT SUBSET EVALS

In [1]:
import pickle
import random

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

from suitability.filter.suitability_efficient import SuitabilityFilter
from suitability.filter.tests import non_inferiority_ttest

# Set seeds for reproducibility
random.seed(32)
np.random.seed(32)

# Configuration
data_name = "rxrx1"
root_dir = "/mfsnic/projects/suitability/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
algorithm = "ERM"
model_type = "last"
seeds = [0, 1, 2]


for seed in seeds:
# Load the features
    feature_cache_file = (
        f"suitability/results/features/{data_name}_{algorithm}_{model_type}_{seed}.pkl"
    )
    with open(feature_cache_file, "rb") as f:
        full_feature_dict = pickle.load(f)

    # Load the split indices
    split_cache_file = f"suitability/results/split_indices/{data_name}.pkl"
    with open(split_cache_file, "rb") as f:
        split_dict = pickle.load(f)

    all_keys = list(split_dict.keys())
    for key in all_keys:
        if key[0] == 'test' or key[0] == 'val':
            del split_dict[key]

    print(split_dict.keys())

    # Define suitability filter and experiment parameters
    classifiers = [
        "logistic_regression"
    ]  # "logistic_regression", "svm", "random_forest", "gradient_boosting", "mlp", "decision_tree"]
    margins = [0, 0.005, 0.01, 0.05]
    normalize = True
    calibrated = True
    sf_results = []
    direct_testing_results = []
    feature_subsets = [
        [0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9],
        [10],
        [11],
        # [4, 11],
        # [4, 11, 8],
        # [4, 11, 8, 6],
        # [4, 11, 8, 6, 2],
        # [4, 11, 8, 6, 2, 1],
        # [4, 11, 8, 6, 2, 1, 0],
        # [4, 11, 8, 6, 2, 1, 0, 7],
        # [4, 11, 8, 6, 2, 1, 0, 7, 3],
        # [4, 11, 8, 6, 2, 1, 0, 7, 3, 10],
        # [4, 11, 8, 6, 2, 1, 0, 7, 3, 10, 5],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    ]
    num_fold_arr = [15]


    # Main loop
    for user_split_name, user_filter in tqdm(split_dict.keys()):
        print(f"Evaluating user split: {user_split_name} with filter {user_filter}")

        # Get user split indices
        user_split_indices = split_dict[(user_split_name, user_filter)]

        # Get user split features and correctness
        all_features, all_corr = full_feature_dict[user_split_name]
        user_features = all_features[user_split_indices]
        user_corr = all_corr[user_split_indices]
        user_size = len(user_corr)
        user_acc = np.mean(user_corr)

        # Re-partition remaining data into folds
        remaining_indices = np.setdiff1d(np.arange(len(all_corr)), user_split_indices)
        source_features = all_features[remaining_indices]
        source_corr = all_corr[remaining_indices]

        for num_folds in num_fold_arr:
            source_fold_size = len(source_corr) // num_folds
            indices = np.arange(len(source_corr))
            np.random.shuffle(indices)
            fold_indices = [
                indices[i * source_fold_size : (i + 1) * source_fold_size]
                for i in range(num_folds)
            ]

            for i, reg_indices in enumerate(fold_indices):
                reg_features = source_features[reg_indices]
                reg_corr = source_corr[reg_indices]
                reg_size = len(reg_corr)
                reg_acc = np.mean(reg_corr)

                for j, test_indices in enumerate(fold_indices):
                    if i == j:
                        continue
                    test_features = source_features[test_indices]
                    test_corr = source_corr[test_indices]
                    test_size = len(test_corr)
                    test_acc = np.mean(test_corr)

                    for classifier in classifiers:
                        for feature_subset in feature_subsets:
                            suitability_filter = SuitabilityFilter(
                                test_features,
                                test_corr,
                                reg_features,
                                reg_corr,
                                device,
                                normalize=normalize,
                                feature_subset=feature_subset,
                            )
                            suitability_filter.train_classifier(
                                calibrated=calibrated, classifier=classifier
                            )

                            for margin in margins:
                                # Test suitability filter
                                sf_test = suitability_filter.suitability_test(
                                    user_features=user_features, margin=margin
                                )
                                p_value = sf_test["p_value"]
                                ground_truth = user_acc >= test_acc - margin

                                sf_results.append(
                                    {
                                        "data_name": data_name,
                                        "algorithm": algorithm,
                                        "seed": seed,
                                        "model_type": model_type,
                                        "normalize": normalize,
                                        "calibrated": calibrated,
                                        "margin": margin,
                                        "reg_fold": i,
                                        "reg_size": reg_size,
                                        "reg_acc": reg_acc,
                                        "test_fold": j,
                                        "test_size": test_size,
                                        "test_acc": test_acc,
                                        "user_split": user_split_name,
                                        "user_filter": user_filter,
                                        "user_size": user_size,
                                        "user_acc": user_acc,
                                        "p_value": p_value,
                                        "ground_truth": ground_truth,
                                        "classifier": classifier,
                                        "feature_subset": feature_subset,
                                        "acc_diff": user_acc - test_acc,
                                        "acc_diff_adjusted": user_acc + margin - test_acc,
                                    }
                                )

                                # Run non-inferiority test on features directly
                                if (
                                    len(feature_subset) == 1
                                    and margin == 0
                                    and classifier == "logistic_regression"
                                    and (j == 0 or (i == 0 and j == 1))
                                ):
                                    test_feature_subset = test_features[:, feature_subset].flatten()
                                    user_feature_subset = user_features[:, feature_subset].flatten()
                                    test_1 = non_inferiority_ttest(
                                        test_feature_subset,
                                        user_feature_subset,
                                        increase_good=True,
                                    )
                                    test_2 = non_inferiority_ttest(
                                        test_feature_subset,
                                        user_feature_subset,
                                        increase_good=False,
                                    )
                                    direct_testing_results.append(
                                        {
                                            "data_name": data_name,
                                            "algorithm": algorithm,
                                            "seed": seed,
                                            "model_type": model_type,
                                            "test_fold": j,
                                            "test_size": test_size,
                                            "test_acc": test_acc,
                                            "user_split": user_split_name,
                                            "user_filter": user_filter,
                                            "user_size": user_size,
                                            "user_acc": user_acc,
                                            "p_value_increase_good": test_1["p_value"],
                                            "p_value_decrease_good": test_2["p_value"],
                                            "ground_truth": ground_truth,
                                            "feature_subset": feature_subset,
                                            "acc_diff": user_acc - test_acc,
                                        }
                                    )

    # Save results
    sf_evals = pd.DataFrame(sf_results)
    sf_evals.to_csv(
        f"suitability/results/sf_evals/erm/rxrx1_sf_results_id_subsets_{algorithm}_{model_type}_{seed}.csv",
        index=False,
    )
    direct_testing_evals = pd.DataFrame(direct_testing_results)
    direct_testing_evals.to_csv(
        f"suitability/results/sf_evals/erm/rxrx1_direct_testing_results_id_subsets_{algorithm}_{model_type}_{seed}.csv",
        index=False,
    )


dict_keys([('id_test', "{'cell_type': 'HEPG2'}"), ('id_test', "{'cell_type': 'HUVEC'}"), ('id_test', "{'cell_type': 'RPE'}"), ('id_test', "{'cell_type': 'U2OS'}")])


  0%|                                                                                              | 0/4 [00:00<?, ?it/s]

Evaluating user split: id_test with filter {'cell_type': 'HEPG2'}


 25%|█████████████████████▎                                                               | 1/4 [01:47<05:21, 107.24s/it]

Evaluating user split: id_test with filter {'cell_type': 'HUVEC'}


...

 50%|██████████████████████████████████████████▌                                          | 2/4 [03:41<03:43, 111.54s/it]

Evaluating user split: id_test with filter {'cell_type': 'RPE'}


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [05:29<01:49, 109.51s/it]

Evaluating user split: id_test with filter {'cell_type': 'U2OS'}


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [07:13<00:00, 108.46s/it]
